# 流
流对于增强构建在llm上的应用程序的响应性至关重要。通过渐进式地显示输出，甚至在完整响应准备好之前，流式处理显著改善了用户体验（UX），特别是在处理llm的延迟时。
## 代理进程
要流式传输代理进程，使用stream或aststream方法，并带有stream_mode="updates"。这将在每个代理步骤之后发出一个事件。

例如，如果你有一个代理调用一个工具一次，你应该看到以下更新：

LLM节点：带有工具调用请求的AIMessage

工具节点：带有执行结果的ToolMessage

LLM节点：最终的AI响应

In [1]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
import os
model = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)
def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_bd24e6fd9bcd4526a19016'}]
step: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]
step: model
content: [{'type': 'text', 'text': "It seems like you're referencing the famous quote from *Parks and Recreation*! While that's a fun saying, San Francisco's weather is actually quite variable. It's often foggy and cool, especially in the summer months, with sunny days more common in the fall and winter. Always good to check the forecast before heading out! 😊"}]


## LLM tokens
要在LLM生成token时流式传输token，请使用stream_mode="messages"。下面可以看到代理流工具调用的输出和最终响应。

In [2]:
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

node: model
content: [{'type': 'tool_call_chunk', 'id': 'call_6d793a3535d44b19bd4cdf', 'name': 'get_weather', 'args': '{"city": "', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': '', 'name': None, 'args': 'San Francisco"}', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': '', 'name': None, 'args': None, 'index': 0}]


node: model
content: []


node: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]


node: model
content: [{'type': 'text', 'text': 'That'}]


node: model
content: [{'type': 'text', 'text': "'s great"}]


node: model
content: [{'type': 'text', 'text': ' to'}]


node: model
content: [{'type': 'text', 'text': ' hear! San'}]


node: model
content: [{'type': 'text', 'text': ' Francisco does have a lovely'}]


node: model
content: [{'type': 'text', 'text': ' climate, though it can'}]


node: model
content: [{'type': 'text', 'text': ' be quite variable—sometimes'}]


node: model
content: [{'type': 'text

## 自定义更新
要在工具执行时流式传输更新，可以使用get_stream_writer。

In [5]:
from langgraph.config import get_stream_writer  
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()  
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="custom"
):
    print(chunk)

Looking up data for city: San Francisco
Acquired data for city: San Francisco


如果在工具中添加get_stream_writer，则无法在LangGraph执行上下文之外调用该工具。

## 流式传输多模式
你可以通过传递流模式来指定多个流模式：stream_mode=["updates", "custom"]：

In [6]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)

for stream_mode, chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]
):
    print(f"stream_mode: {stream_mode}")
    print(f"content: {chunk}")
    print("\n")

stream_mode: updates
content: {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 153, 'total_tokens': 173, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-6ebf8bd2-0241-4525-b404-a88d949bfee5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--cc2edf47-816b-4b59-8b43-ae02432b281a-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_acb1a1ae01d14d089c99d2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 20, 'total_tokens': 173, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}}


stream_mode: custom
content: Looking up data for city: San Francisco


stream_mode: custom
content: Acquired data for city: San Francisco


stream_mode:

## 禁用流
在某些应用程序中，您可能需要禁用给定模型的单个令牌流。
这在多智能体系统中非常有用，可以控制哪些智能体输出流。